# Image walkthrough

Tools used:

- ffmpeg
- Python (PIL, Numpy)

## Camera



In [ ]:
!dir "WL357718\Canon DIGITAL IXUS 70\"

In [ ]:
!dir D:\DCIM\100___11\*.mp4

## ffmpgeg

With ffmpeg one can convert different picture or video formats.

- Extract frames from video https://trac.ffmpeg.org/wiki/Create%20a%20thumbnail%20image%20every%20X%20seconds%20of%20the%20video
- Concat frames to video https://trac.ffmpeg.org/wiki/Slideshow

`"c:\Users\jokemjaa\Downloads\ffmpeg-20190705-a514244-win64-static\ffmpeg-20190705-a514244-win64-static\bin\ffmpeg" -i frame%04d.png out.mp4`

## Numpy

Numpy tarjoaa hyviä funktioita, joilla voi aggregoida. https://docs.scipy.org/doc/numpy/reference/routines.statistics.html

- median
- max
- min 
- percentile
- quantile
- mean
- average



## Filesystem

- 'input' has videos
- 'temp' has temporary files
- 'output' has output images

In [ ]:
!dir c:\work\input\*.MP4

## Dev

In [ ]:
import subprocess
import stillvideos as sv
import importlib
importlib.reload(sv)
import numpy as np
import scipy
from skimage.color import rgb2hsv
from skimage.color import hsv2rgb

# sv.iterMovie()
# sv.inputMovies()
# sv.sdCardMovies()
# sv.copyFromSdCard()
# sv.newSdCardMovies()

Tuodaan tuoreimman käsittelemättömän videon freimit input kansioon

In [ ]:
# inputVideo = sv.iterMovie(filetype = "MOV")
# inputVideo
inputVideo = 'MVI_0212.MP4'
sv.extractFrames(inputVideo)

pageName = inputVideo.split(".")[0]

In [ ]:
transpose = False
# sv.generateThumbnails(pageName, range(0, 99999, 30), factor = 0.5, transpose = transpose)

In [ ]:
sv.generateThumbnails(pageName, skip = 50)

In [ ]:
sv.openHtmlPage()

In [ ]:
beatsPerInterval = 8
beatsPerMinute = 60.0
beatsPerSecond = beatsPerMinute / 60.0
framesPerBeat = 30 / beatsPerSecond
framesPerInterval = framesPerBeat * beatsPerInterval
framesPerInterval

In [ ]:
framesPerInterval = 50
start = 180
copyCount = 3
end = framesPerInterval * copyCount + start

choice = range(start, int(end), 1)

In [ ]:
importlib.reload(sv)
imMedian = sv.tempMedian(choice)
imMedian

In [ ]:
# weights = [i for i in range(1,11)] + [i for i in reversed(range(1,11))]
weights = [np.sin((0.5 + i) * np.pi / 10 ) ** 2 for i in range(10)]
weights

In [ ]:
importlib.reload(sv)
import numpy as np
# choice = range(390, 1710, 1)
offset = 0
atol = 0.1
# copyCount = 10
im = sv.sumCopies(choice, imMedian, copyCount, offset, atol, 0)
# im = sv.avgCopies(choice, imMedian, copyCount, offset, np.quantile, q = 0.9)
# im = sv.avgCopies(choice, imMedian, copyCount, offset, np.average, weights = weights)
im

In [ ]:
im = sv.readTempFolderImages([100])[0][0]

importlib.reload(sv)
sv.g.verbose = False
# imc = sv.getMaskImg(imMedian, im, im2, 0.0, 0.4)
im

In [ ]:
np.mean(np.array(im), axis = 0)

In [ ]:
np.apply_over_axes(np.mean, np.array(im), [0,1])


In [ ]:
np.apply_over_axes(np.percentile, np.array(im), )


In [ ]:
np.percentile(np.array(im), q = 10, axis = [0, 1])

In [ ]:

hue_img = rgb2hsv(im)

In [ ]:
def asImg(nparray):
#     print(nparray)
#     display(Image.fromarray(nparray))
    return Image.fromarray(nparray)

In [ ]:
im = sv.readTempFolderImages([250])[0][0]

def myTrans(im, hue_offset = 0.5):
    im_ret = Image.new(im.mode, im.size, (0, 0, 0))
    im2 = Image.new(im.mode, im.size, (255, 255, 255))
    imc = sv.getMaskImg(imMedian, im, im2, 0.0, 0.2)
    imv = scipy.ndimage.binary_opening(np.array(imc)[:,:,2])
    imv = scipy.ndimage.binary_closing(imv)
#     Image.fromarray(imv.astype(np.uint8) * 255)
    imn = np.array(im) 
    # imn[:,:,0][imn[:,:,0] < 100] = 100
    im_hue = rgb2hsv(imn)
    im_hue[:,:,0][imv > 0] = im_hue[:,:,0][imv > 0] + hue_offset
    # im_hue[:,:,1][im_hue[:,:,1] < 0.01] = 0.01
    # im_hue[:,:,2][im_hue[:,:,2] > 0.8] = 0.8
    im_ret = (hsv2rgb(im_hue) * 255).astype(np.uint8)
    return(asImg(im_ret))

myTrans(im, 0.2)

In [ ]:
from skimage.filters import gaussian
from PIL import Image
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
im = sv.readTempFolderImages([250])[0][0]
from skimage.morphology import square

def myDif(im1, im2, atol, rtol, sigma):
#     np_im1 = gaussian(np.asarray(im1), sigma)
#     np_im2 = gaussian(np.asarray(im2), sigma)
    np_im1 = np.asarray(im1)
    np_im2 = np.asarray(im2)
    np_dif_1 = np.isclose(np_im1[:,:,0], np_im2[:,:,0], atol = atol, rtol = rtol)
    np_dif_2 = np.isclose(np_im1[:,:,1], np_im2[:,:,1], atol = atol, rtol = rtol)
    np_dif_3 = np.isclose(np_im1[:,:,2], np_im2[:,:,2], atol = atol, rtol = rtol)
    #     print(np_dif)
#     print(np_im1.shape)
    np_dif = np.logical_and(np_dif_1, np_dif_2, np_dif_3)
#     np_dif = closing(np_dif, square(6))
#     im2 = Image.new(im.mode, im.size, (255, 255, 255))
#     imr = sv.getMaskImg(imMedian, im, im2, 0.0, 0.2)
#     return imr
#     np_dif = scipy.ndimage.binary_opening(np.array(np_dif))
#     np_dif = scipy.ndimage.binary_closing(np.array(np_dif))
#     return(asImg(imr.astype(np.uint8) * 255))
    np_ret = np_dif.astype(np.uint8) * 255
    return asImg(np_ret)

imCur = myDif(im, imMedian, 0.0, 0.06, 1)

In [ ]:
no1, no2 = 250, 251

temps = range(240,260,2)
ret = None
for i in temps:
    print(i)
    im1 = sv.readTempFolderImages([i])[0][0]
    im2 = imMedian
    if ret is None:
        ret = np.array(myDif(im1, im2, 0.0, 0.02, 3))
    else:
        ret = np.append(ret, np.array(myDif(im1, im2, 0.0, 0.04, 1))) 

In [ ]:
ret.shape

In [ ]:
ret = ret.reshape(10, 720, 1280)
myav = np.mean(ret, axis = 0)
myav.shape
asImg(myav.astype(np.uint8))
# np.mean(myav)

In [ ]:
asImg(opening((255 - np.array(imCur)).astype(np.uint8)))

In [ ]:
npCur = 255 - np.array(imCur)
npCur[myav < 90] = 0
asImg(npCur)

In [ ]:
np.mean(np.array(imCur))
np.mean(myav)

In [ ]:
myDif(im, imMedian, 0.0, 0.02, 2)

In [ ]:
from PIL import Image
im = Image.open('G:\jokemjaaPublic\misc\mikki.jpg')

In [ ]:
im.size, im.format

In [ ]:
im2 = im.resize((200, 355))

In [ ]:
im2.size

In [ ]:
sum(np.array(imc))

In [ ]:
nyBlur = ImageFilter.BoxBlur(2)
imb = imc.filter(nyBlur)
threshold = 200  
imc = imb.point(lambda p: p > threshold and 255) 
imc

In [ ]:
import matplotlib.pyplot as plt
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(14, 5))

ax0.imshow(im)
ax0.set_title("RGB image")
ax0.axis('off')
ax1.imshow(hue_img, cmap='hsv')
ax1.set_title("Hue channel")
ax1.axis('off')

fig.tight_layout()
plt.show()

In [ ]:
sv.getMask(imMedian, im, 0)

In [ ]:
offset = 0
weights1 = [np.sin((i - 1.0 * offset / outputFrameCount) * np.pi / copyCount ) ** 2 for i in range(copyCount)]
offset = 119
weights2 = [np.sin((i + 1.0 * offset / outputFrameCount) * np.pi / copyCount ) ** 2 for i in range(copyCount)]

print(weights1, weights2)
# im = sv.avgCopi§es(choice, imMedian, copyCount, offset, np.average, weights = weights)
# im 

In [ ]:
test = [i for i in range(960)]
offset = 0
fuuga1 = [offset + i * framesPerInterval for i in range(copyCount)]
offset = 119
fuuga2 = [offset + i * framesPerInterval for i in range(copyCount)]
fuuga1, fuuga2

In [ ]:
copyCount = 2
choice = range(100, 280)
framesPerInterval = 1
print(f"""
choice              = {choice}
framesPerInterval   = {framesPerInterval}
copyCount           = {copyCount}

""")
outputFrameCount = int(framesPerInterval)


In [ ]:

import numpy as np
importlib.reload(sv)

sv.clearOutputFolder()
transpose = False
atol = 0
rtol = 0.1

outputFrameCount = 90
start = 100

for offset in range(outputFrameCount):
    imFile = f"c:/work/output/frame{offset:04d}.png"
    print(imFile)
    imIn = sv.readTempFolderImages([start + offset])[0][0]
#     im = myTrans(imIn, offset / outputFrameCount * 5.0)
#     im = sv.sumCopies(choice, imMedian, copyCount, offset, atol, rtol)
#    im = sv.avgCopies(choice, imMedian, copyCount, offset, np.quantile, q = 0.9)
    weights = [np.sin((i + 1.0 * offset / outputFrameCount) * np.pi /copyCount ) ** 2 for i in range(copyCount)]
    print(weights)
    im = sv.avgCopies(choice, copyCount, offset, np.average, weights = weights)
    if transpose:
        im = im.transpose(Image.ROTATE_270)
    im.save(imFile)
    
# display(Image.fromarray(imMasked))

In [ ]:
importlib.reload(sv)
sv.concatFrames('koski2.mp4')

In [ ]:
import sys
# C:\Users\jokemjaa\Documents\GitHub\jupyter\jedit
sys.path.append("C:\\users\\jokemjaa\\Documents\\github\\")
import jupyter_cell_magics.jedit

In [ ]:
%%jedit -w video_player.css -t video_player.css -t video_player.html -o temp.html
<style>
* {
  box-sizing: border-box;
}

body {
  margin: 0;
  font-family: Arial;
  font-size: 17px;
}

#myVideo {
  position: fixed;
//  left: 6%;
//  top: 10%;
//  min-width: 60%; 
  min-height: 100%;
    width 2024px;
}

.content {
  position: fixed;
  bottom: 0;
  background: rgba(0, 0, 0, 0.5);
  color: #f1f1f1;
  width: 80%;
  padding: 20px;
}

#myBtn {
  width: 200px;
  font-size: 18px;
  padding: 10px;
  border: none;
  background: #000;
  color: #fff;
  cursor: pointer;
}

#myBtn:hover {
  background: #ddd;
  color: black;
}
</style>


In [ ]:
%%jedit -w video_player.html -o temp.html
{% include 'video_player.css' %}
<video autoplay muted loop id="myVideo">
  <source src="c:/work/output/autumncolors.mp4" type="video/mp4">
  Your browser does not support HTML5 video.
</video>

<div class="content">
  <h1>Ariel</h1>
  <p>Maauimalan takana olevassa metsassa</p>
  <button id="myBtn" onclick="myFunction()">Pause</button>
    <p id = "tiedot"></p>
<div class="slidecontainer">
  <input type="range" min="1" max="100" value="50" class="slider" id="myRange">
  <p>Value: <span id="demo"></span></p>
</div>
</div>

<script>
var slider = document.getElementById("myRange");
var output = document.getElementById("demo");

slider.oninput = function() {
  output.innerHTML = this.value;
    video.currentTime = video.duration * this.value / 100;
}
</script>

<script>
var video = document.getElementById("myVideo");
var btn = document.getElementById("myBtn");
var tiedot = document.getElementById("tiedot");

function myFunction() {
  if (video.paused) {
    video.play();
    btn.innerHTML = "Pause";
  } else {
    video.pause();
    btn.innerHTML = "Play";
    tiedot.innerHTML = video.currentTime;
  }
}
</script>


In [ ]:
copyCount = 5
offset = 0

with sv.tempFolderImages(choice) as ims:
    imageCount = int(len(ims) / copyCount)
    print(f"Image count {imageCount}")
    for i in range(copyCount):
        display(ims[offset + i * imageCount][0])

In [ ]:
choice = range(0, 640, 1)
from PIL import Image
with sv.tempFolderImages(choice) as ims:
    ret = []
    for ind, imRaw in enumerate(ims):
        im = imRaw[0]#.transpose(Image.ROTATE_270)
        ret.append(np.array(im)[ind])
#         ret.append(np.array(im)[2 * ind + 1])
Image.fromarray(np.array(ret))

## Keskiarvokuvat

Keskiarvokuvat tehdään Numpyn funktioilla. Muista `transpose`, jos kuva pitää kääntää.

In [ ]:
import numpy as np

In [ ]:
# choice = range(200, 1800, 2)
choice = range(100, 1000, 3)
transpose = False
tests = [("Kirkkaampi kuin 50 prosenttia pikseleistä", np.quantile, {'q' : 0.5})
         ,("Kirkkaampi kuin 90 prosenttia pikseleistä", np.quantile, {'q' : 0.9})
         ,("Pikselien keskiarvo", np.average, {})
         ,("Kirkkaampi kuin 10 prosenttia pikseleistä", np.quantile, {'q' : 0.10})
#          ,("Kirkkaampi kuin  0 prosenttia pikseleistä", np.quantile, {'q' : 0.0})
#          ,("Keskihajonta", myStd, {})
        ]

sv.generateAggregates(pageName, tests, choice, transpose)


In [ ]:
np.mean(npDiffAv)

In [ ]:
np.mean(npDiff1)

In [ ]:
f"file:///C:/work/html/{pageName}/page.html"

In [ ]:
%%html
<a href = "file:///C:/work/html/MVI_2126/page.html">linkki<a/>

In [ ]:
%%writefile c:/work/html/pageTemplate.html
<!DOCTYPE html>
<html>
<body>

<<<code>>>

</body>
</html>


In [ ]:
Image.fromarray(np.array(np.round(out), dtype=np.uint8))

In [ ]:
# imlist[-2]
# out.filter(myFilter)
Image.fromarray(out)

In [ ]:
out.save("C:/Users/jokemjaa/OneDrive - Tieto Corporation/Notebooks/average_6.png")